![logo.png](./img/logo.png)

## Open Policy Agent

A general purpose policy engine

With a declarative language: Rego

### Background

Started by Styra in late 2015

![commits.png](./img/commits.png)

Current version: 1.15.1

Adopted into the Cloud Native Computing Foundation in March 2018

Currently in the Incubating stage

In the Assess stage of Tech Radar (73)

### General workings

Most of the following from Red Hat's blog [here](https://servicesblog.redhat.com/2019/10/16/open-policy-agent-part-i-the-introduction/).

![flow.png](./img/flow.png)

In [1]:
ls src/

acl.json  policy.rego


#### Our access control list:

In [2]:
cat src/acl.json

{
  "nathan": [
    "read"
  ],
  "bryan": [
    "read"
  ],
  "courtney": [
    "read",
    "write"
  ],
  "kurt": [
    "read"
  ]
}

#### Our policy definition:

In [3]:
cat src/policy.rego

package myapi.policy

import data.myapi.acl
import input

default allow = false

allow {
        access = acl[input.user]
        access[_] == input.access
}

whocan[user] {
        access = acl[user]
        access[_] == input.access
}

#### The cli:

In [4]:
opa

An open source project to policy-enable your service.

Usage:
  opa [command]

Available Commands:
  build       Compile Rego policy queries
  check       Check Rego source files
  deps        Analyze Rego query dependencies
  eval        Evaluate a Rego query
  fmt         Format Rego source files
  help        Help about any command
  parse       Parse Rego source file
  run         Start OPA in interactive or server mode
  test        Execute Rego test cases
  version     Print the version of OPA

Flags:
  -h, --help   help for opa

Use "opa [command] --help" for more information about a command.


#### Start up our server:

In [7]:
opa run --server &

[1] 15815


: 1

#### Upload our acl and policy:

In [8]:
curl -X PUT http://localhost:8181/v1/data/myapi/acl \
--data-binary @src/acl.json

<html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>openresty/1.15.8.1</center>
</body>
</html>


In [ ]:
curl -X PUT http://localhost:8181/v1/policies/myapi \
--data-binary @src/policy.rego

#### And test:

In [ ]:
curl -s -X POST http://localhost:8181/v1/data/myapi/policy/allow \
--data-binary '{ "input": { "user": "courtney", "access": "write" } }' | jq

#### Cleanup:

In [ ]:
pkill opa

## Other uses

Adding user auth to the Docker daemon ([link](https://www.openpolicyagent.org/docs/latest/docker-authorization/))

Adding a policy to Terraform ([link](https://www.openpolicyagent.org/docs/latest/terraform/))

Only allowing images from a trusted registry in Kubernetes

### Future

Gatekeeper v3 in beta currently

WebAssembly compiler going GA in v0.16